In [3]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

## read data 
CIP_data = pd.read_csv("CIP_data_encoded.csv")

In [5]:
### Step 1: create model and calculate apparent performance metric of interest (P)
X = CIP_data[['ANC', 'ATL', 'BAL', 'BHM', 'BOS', 'BUF', 'CAM', 'CHI',
       'CIN', 'CLE', 'COL', 'DAL', 'DEN', 'DTR', 'FBG', 'GRB', 'HON', 'IND',
       'JAC', 'KCY', 'LAX', 'LBC', 'LVG', 'MIA', 'MIL', 'MIN', 'NOR', 'NYC',
       'OKC', 'ORA', 'PHI', 'PHX', 'PON', 'POR', 'RIC', 'SDG', 'SEA', 'SFO',
       'SLC', 'STL', 'WDC', 'MSMW', 'MSW', 'Oth/Unk/Missing']]

y = CIP_data['Susceptible']


Random_forest_model = RandomForestClassifier(n_estimators = 100)  


Random_forest_model_fitted = Random_forest_model.fit(X, y)

y_predict = Random_forest_model_fitted.predict(X)


ROC_AUC_random_forest = metrics.roc_auc_score(y, y_predict)
print(ROC_AUC_random_forest) # 0.5 again...
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y, y_predict)) ## but get model accuracy of 0.9470427693866846... so why is the ROC so low? 


0.5
ACCURACY OF THE MODEL:  0.9470427693866846
